> FIAP - Faculdade de Informática e Administração Paulista

# Global Solutions 2023 - AI Challenges and Solutions

Detector de alimentos usando YOLOv5 e ESP32-CAM


## Criação do banco de imagens de alimentos

Para treinar o nosso detector, precisamos de um bom número de exemplos de imagens dos alimentos que queremos detectar, para isso, vamos usar imagens da internet contendo o nosso alimento. A ideia é ter o máximo de tipos de fotos de diversos ângulos do nosso alimento.

Quanto mais exemplos e situações temos do nosso alimento (Ambiente escuro, ângulos, fases de maturação, etc.) melhor nosso modelo vai detectar. Por exemplo, se tivemos apenas exemplos de fotos de perfil de um alimento, ele provavelmente só irá conseguir detectar um alimento se colocarmos ele de perfil e com uma iluminação focada no alimento.


### Busca de fotos

- Procure imagens contendo o alimento e salve tudo em uma pasta local.
  - Se possível, renomeie cada arquivo nesse formato: `[NOME DO ALIMENTO]_[NUMERO DO EXEMPLO]` (Opcional)
  - Exemplo: `maca_01`. Isso vai facilitar na visualização e organização dos arquivos.
- Após isso, separe 80% em uma nova pasta chamada `train` e o resto em uma pasta chamada `val`
- Mova as pastas `train` e `val` para uma nova pastas chamada `images`

### Criação dos labels

Agora que temos as imagens separadas, vamos criar os rótulos de cada alimento, para isso, temos que delimitar onde cada alimento está em cada imagem de treino e validação.

Vamos usar o site [makesense.ai](https://makesense.ai) para isso.

- Entre no site e clique em "Get started"
- Arraste a pasta de imagens de treino/validacao para o quadrado das imagens. (Você irá precisar fazer esse processo com as duas, uma de cada vez)
- Clique em "Object Detection"
- No modal de "Create Labels" crie um rótulo para cada tipo de alimento que você irá rotular.
- Clique em "Start project"
- Crie uma caixa delimitadora retangular para cada alimento em cada imagem. Exemplo:
  ![Exemplo label de uma maçã](docs/readme/label.png)
  > Importante! Se houver mais de um alimento na foto, crie uma caixa para cada alimento, mesmo que eles sejam do mesmo tipo. Exemplo: Se houver duas maçãs na foto, crie uma caixa para cada maçã, e não uma caixa só para as duas junto.
- Após criar as caixas para todas as imagens, vá em "Actions" e "Export annotations".
- Dentro do modal, selecione "A .zip package containing files in YOLO format". Dentro desse zip estão todos os rótulos para cada imagem.
- Crie uma pasta chamada `labels` e dentro dela crie uma nova pasta com o tipo de imagem que você estava rotulando (`train` ou `val`).
- Extraia o conteúdo do .zip para a pasta que você criou.
- Repita o processo com as imagens de validação/treino. No final, você terá que ter os labels de treino e validação dentro da sua respectiva pasta.

No final do processo, você terá que ter essa estrutura de arquivos:

```text
- images/
  - train/
    - imagens de treino
  - val/
    - imagens de validação
- labels/
  - train/
    - labels das imagens de treino
  - val/
    - labels das imagens de validação
```

**Pronto! Agora vamos para o treinamento do nosso modelo**


## Treinando o modelo

### Configurando o YOLO

Primeiro, faça o clone ou baixe como zip e extraia o repositório oficial do YOLOv5:

- [Download como zip](https://github.com/ultralytics/yolov5/archive/refs/heads/master.zip)
- Ou rode execute o código abaixo:


In [ ]:
! git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16008, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 16008 (delta 22), reused 21 (delta 13), pack-reused 15967
Receiving objects: 100% (16008/16008), 14.60 MiB | 19.19 MiB/s, done.
Resolving deltas: 100% (10988/10988), done.


Com o código do YOLO no seu computador, baixe as dependências necessárias para fazer ele rodar:


In [ ]:
! pip install -r yolov5/requirements.txt  # install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Crie um arquivo de configuração para o dataset chamado `alimentos.yaml`, contendo o seguinte código:

```yaml
train: ../../images/train/ # NÃO EDITAR
val: ../../images/val/ # NÃO EDITAR

names: # Defina aqui as labels do seu dataset
  0: "Maçã"
  1: "Banana"
  2: "Laranja"
  # ...
```

As labels tem que estar na mesma ordem que você criou no makesense.ai

Com o arquivo de configuração criado, podemos treinar o modelo.


### Treinando o modelo

Rode o código abaixo para treinar o modelo:


In [ ]:
! python yolov5/train.py --data trator.yaml --weights yolov5l.pt --img 640 --epochs 40

train: weights=yolov5l.pt, cfg=, data=trator.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-227-ge4df1ec Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.

Quando terminar de executar, ele irá mostrar o caminho de onde ficaram os resultados, geralmente: `yolov5/runs/train/exp[...]`

Dentro dessa pasta, você consegue ver várias informações sobre o treinamento, como os pesos, os gráficos de perda, etc.

Copie o caminho do arquivo `best.pt` dentro da pasta `weights` para podermos usar posteriormente.


### Testando o modelo

Vamos rodar o nosso modelo para ver como ele se comporta.

Primeiro, crie uma pasta chamada `tests` e coloque dentro imagens que você deseja testar, de preferência imagens que não foram usadas para treinar o modelo.

Com as imagens na pasta, rode o código abaixo para testar o modelo:


In [ ]:
import os
import subprocess

def get_latest_train_run_folder():
    subfolders = [f.path for f in os.scandir('yolov5/runs/train') if f.is_dir()]
    latest_folder = max(subfolders, key=os.path.getctime, default=None)
    return latest_folder

latest_run = get_latest_train_run_folder()

# COMANDO

result = subprocess.run(f'python yolov5/detect.py --weights {latest_run}/weights/best.pt --img 640 --source tests/ --data yolov5/data/trator.yaml', shell=True)

Na linha final, o YOLO mostra onde ficou salvo os resultados da seguinte maneira:

`Results saved to yolov5/runs/detect/exp1` (O número da pasta varia de acordo com o número de vezes que o YOLO foi executado)

Dentro dessa pasta ele gera uma imagem com a box de detecção para cada imagem de teste. Exemplo:

![Imagem de teste com label gerada pelo YOLO](docs/readme/test.jpg)


## Conexão com o ESP32-CAM

Primeiro, rode o [código](https://docs.google.com/document/d/1AnM97ejIS-_6f1mqPYu6Kl_hdNWh-9GAqmaCRdsK8W0/edit) que disponibiliza as imagens do ESP32 no Arduino IDE. Copie o link que ele gera no monitor serial para usar posteriormente.

Com o código rodando, faça as alterações necessárias no código abaixo (linhas 6 e 7) para que ele se conecte ao ESP32-CAM e reconheça as imagens que ele envia.


In [ ]:
import cv2
import torch
import numpy as np
import urllib

path = f'yolov5/runs/train/exp/weights/best.pt' # (OPCIONAL) TROQUE PELO CAMINHO DO SEU PESO CASO QUEIRA (best.pt que foi gerado no treinamento) ex: yolov5/runs/train/exp9/weights/best.pt
image_url = 'http://192.168.43.201/cam-lo.jpg' # TROQUE PELO LINK GERADO NO MONITOR SERIAL

model = torch.hub.load('ultralytics/yolov5', 'custom', path, force_reload=True)

print(path)

while True:
    img_resp=urllib.request.urlopen(url=image_url)
    imgnp=np.array(bytearray(img_resp.read()),dtype=np.uint8)
    im = cv2.imdecode(imgnp,-1)

    results = model(im)

    print(results)

    frame = np.squeeze(results.render())

    cv2.imshow('Deteccao', frame)

    key=cv2.waitKey(5)

    if key==ord('q'):
        break

cv2.destroyAllWindows()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-10-16 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


yolov5/runs/train/exp/weights/best.pt


URLError: ignored

Ao rodar o código, deverá abrir uma janela mostrando a camera do ESP-32. Aponte a camera para um alimento que você treinou e veja o resultado.

![Resultado](docs/readme/esp32.png)


# Teste de Webcam

In [ ]:
import cv2
import torch
import numpy as np

path = 'yolov5/runs/train/exp/weights/best.pt'  # (OPCIONAL) TROQUE PELO CAMINHO DO SEU PESO CASO QUEIRA (best.pt que foi gerado no treinamento) ex: yolov5/runs/train/exp9/weights/best.pt

# Abra a câmera padrão (0 para a primeira câmera)
cap = cv2.VideoCapture(0)

model = torch.hub.load('ultralytics/yolov5', 'custom', path, force_reload=True)

while True:
    # Captura um quadro da webcam
    ret, frame = cap.read()

    results = model(frame)

    frame = np.squeeze(results.render())

    cv2.imshow('Detecção', frame)

    key = cv2.waitKey(1)

    if key == ord('q'):
        break

# Libera a captura da câmera e fecha a janela
cap.release()
cv2.destroyAllWindows()
